In [3]:
import intrepid as ip
import intrepid.utils
import imp

f, filename, description = imp.find_module('traffic_light')
traffic_light = imp.load_module('TrafficLight', f, filename, description)

ctx = ip.mk_ctx()

traffic_light.mk_circuit(ctx)
light_out = traffic_light.outputs['traffic_light/out']
targetGreen = ip.mk_eq(ctx, light_out, traffic_light.constants['traffic_light/Green'])
targetYellow = ip.mk_eq(ctx, light_out, traffic_light.constants['traffic_light/Yellow'])
targetRed = ip.mk_eq(ctx, light_out, traffic_light.constants['traffic_light/Red'])
targetOff = ip.mk_eq(ctx, light_out, traffic_light.constants['traffic_light/Off'])

bmc = ip.mk_engine_bmc(ctx)
ip.bmc_add_target(ctx, bmc, targetGreen)
ip.bmc_add_target(ctx, bmc, targetYellow)
ip.bmc_add_target(ctx, bmc, targetRed)
ip.bmc_add_target(ctx, bmc, targetOff)

for depth in range(4):
    reached_target = ip.utils.bmc_reach_at_depth(ctx, bmc, depth)
    if len(reached_target) > 0:
        print 'Targets reached at depth', depth, ':', len(reached_target)
        cex = ip.bmc_get_counterexample(ctx, bmc, reached_target[0])
        cex_dict = ip.utils.counterexample_get_as_dictionary(ctx, cex, traffic_light.inputs, {})
        print cex_dict
    else:
        print 'No target reacheable at depth:', depth

ip.del_ctx(ctx)

Targets reached at depth 0 : 1
{'OnOff': ['?'], 'OnYellow': ['?'], 'OnRed': ['?'], 'Daytime': ['?'], 'OnGreen': ['?']}
Targets reached at depth 1 : 1
{'OnOff': ['?', '?'], 'OnYellow': ['false', 'true'], 'OnRed': ['?', '?'], 'Daytime': ['false', '?'], 'OnGreen': ['?', '?']}
Targets reached at depth 2 : 1
{'OnOff': ['?', '?', '?'], 'OnYellow': ['?', 'false', 'true'], 'OnRed': ['?', '?', '?'], 'Daytime': ['?', 'false', '?'], 'OnGreen': ['?', '?', '?']}
Targets reached at depth 3 : 1
{'OnOff': ['?', '?', '?', '?'], 'OnYellow': ['false', 'true', '?', '?'], 'OnRed': ['?', 'false', 'true', 'false'], 'Daytime': ['false', 'true', 'true', '?'], 'OnGreen': ['?', '?', 'false', 'true']}
